In [1]:
!pip install requests

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import requests
import datetime

In [3]:
import base64

In [4]:
client_id = "c060a698be6449279c07f9f4a3c6e952"
client_secret = "92f9c0d577024d3e85ee90482b268e19"

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id =self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return{
            "Authorization":f"Basic {client_creds_b64}" 
        }
    
    def get_token_data(self):
        return {
        "grant_type":"client_credentials"
        }
    
    def perform_auth(self):
        # do a lookup for a token
        # this token is for future requests
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url,data=token_data,headers=token_headers)
        if r.status_code not in range(200,299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token'] # state-like item(they expire with time)
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [11]:
spotify = SpotifyAPI(client_id,client_secret)

In [12]:
spotify.perform_auth()

True

In [13]:
spotify.access_token

'BQCvdhL9wKVCVoM4SMRs6ly8M4aV3_bW5Dz2eW1I2wm3m2znhjIwcT9pRq6U8GyYT4orxrbdcbstZB1qzc4'

In [14]:
spotify.access_token_expires

datetime.datetime(2020, 8, 7, 11, 41, 35, 104735)

In [15]:
spotify.access_token_did_expire

False